In [53]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
import numpy as np
import torch

In [54]:
df_reviews_train = pd.read_csv('C:/Users/kaush/Desktop/aclImdb/train/trainReviews.txt', sep='delimiter', names=['Review'], engine = 'python')
df_scores_train = pd.read_csv('C:/Users/kaush/Desktop/aclImdb/train/IMDBValuesTrain.txt', sep='delimiter', names=['Score'], engine = 'python')

df_reviews_test = pd.read_csv('C:/Users/kaush/Desktop/aclImdb/test/testReviews.txt', sep='delimiter', names=['Review'], engine = 'python')
df_scores_test = pd.read_csv('C:/Users/kaush/Desktop/aclImdb/test/IMDBValuesTest.txt', sep='delimiter', names=['Score'], engine = 'python')

print(df_reviews_train)
print(df_scores_train)

                                                  Review
0      Story of a man who has unnatural feelings for ...
1      Airport '77 starts as a brand new luxury 747 p...
2      This film lacked something I couldn't put my f...
3      Sorry everyone,,, I know this is supposed to b...
4      When I was little my parents took me along to ...
...                                                  ...
24995  Seeing as the vote average was pretty low, and...
24996  The plot had some wretched, unbelievable twist...
24997  I am amazed at how this movie(and most others ...
24998  A Christmas Together actually came before my t...
24999  Working-class romantic drama from director Mar...

[25000 rows x 1 columns]
       Score
0          3
1          4
2          4
3          1
4          1
...      ...
24995      9
24996      8
24997     10
24998      8
24999      7

[25000 rows x 1 columns]


In [55]:

df_reviews_train['Review'] = df_reviews_train['Review'].str.lower()   # lowercase the Dataset
df_reviews_test['Review'] = df_reviews_test['Review'].str.lower()

df_reviews_train['Review'] = df_reviews_train['Review'].str.replace(r'[^\w\s]+', '')   # Remove Punctuation from Dataset
df_reviews_test['Review'] = df_reviews_test['Review'].str.replace(r'[^\w\s]+', '')

stop_words = set(stopwords.words('english'))
df_reviews_train['Review'] = df_reviews_train['Review'].apply(lambda review: [i for i in review.split() if i not in stop_words])   # Remove Stopwords
df_reviews_test['Review'] = df_reviews_test['Review'].apply(lambda review: [i for i in review.split() if i not in stop_words])   # Remove Stopwords


print(df_reviews_train)
print(df_reviews_test)

                                                  Review
0      [story, man, unnatural, feelings, pig, starts,...
1      [airport, 77, starts, brand, new, luxury, 747,...
2      [film, lacked, something, couldnt, put, finger...
3      [sorry, everyone, know, supposed, art, film, w...
4      [little, parents, took, along, theater, see, i...
...                                                  ...
24995  [seeing, vote, average, pretty, low, fact, cle...
24996  [plot, wretched, unbelievable, twists, however...
24997  [amazed, movieand, others, average, 5, stars, ...
24998  [christmas, together, actually, came, time, iv...
24999  [workingclass, romantic, drama, director, mart...

[25000 rows x 1 columns]
                                                  Review
0      [mr, costner, dragged, movie, far, longer, nec...
1      [example, majority, action, films, generic, bo...
2      [first, hate, moronic, rappers, couldnt, act, ...
3      [even, beatles, could, write, songs, everyone,...
4    

In [56]:
# Change Review Score to Binary Score
for score in df_scores_train['Score']:
    if score <= 5:
        df_scores_train = df_scores_train.replace(score, 0)
    else:
        df_scores_train = df_scores_train.replace(score, 1)

for score in df_scores_test['Score']:
    if score <= 5:
        df_scores_test = df_scores_test.replace(score, 0)
    else:
        df_scores_test = df_scores_test.replace(score, 1)
print (df_scores_train)
print (df_scores_test)

       Score
0          0
1          0
2          0
3          0
4          0
...      ...
24995      1
24996      1
24997      1
24998      1
24999      1

[25000 rows x 1 columns]
       Score
0          0
1          0
2          0
3          0
4          0
...      ...
24995      1
24996      1
24997      1
24998      1
24999      1

[25000 rows x 1 columns]


TypeError: sequence item 0: expected str instance, list found

In [92]:

for i in (df_reviews_train['Review']):
    ' '.join(i)
    print(i)


['story', 'man', 'unnatural', 'feelings', 'pig', 'starts', 'opening', 'scene', 'terrific', 'example', 'absurd', 'comedy', 'formal', 'orchestra', 'audience', 'turned', 'insane', 'violent', 'mob', 'crazy', 'chantings', 'singers', 'unfortunately', 'stays', 'absurd', 'whole', 'time', 'general', 'narrative', 'eventually', 'making', 'putting', 'even', 'era', 'turned', 'cryptic', 'dialogue', 'would', 'make', 'shakespeare', 'seem', 'easy', 'third', 'grader', 'technical', 'level', 'better', 'might', 'think', 'good', 'cinematography', 'future', 'great', 'vilmos', 'zsigmond', 'future', 'stars', 'sally', 'kirkland', 'frederic', 'forrest', 'seen', 'briefly']
['airport', '77', 'starts', 'brand', 'new', 'luxury', '747', 'plane', 'loaded', 'valuable', 'paintings', 'belonging', 'rich', 'businessman', 'philip', 'stevens', 'james', 'stewart', 'flying', 'bunch', 'vips', 'estate', 'preparation', 'opened', 'public', 'museum', 'also', 'board', 'stevens', 'daughter', 'julie', 'kathleen', 'quinlan', 'son', 'lu

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



25000


In [88]:
count_words = Counter(df_reviews_train['Review'])
total_words = len(df_reviews_train['Review'])
sorted_words = count_words.most_common(total_words)
print(count_words)

TypeError: unhashable type: 'list'

[[   11    49  7487 ...     0     0     0]
 [ 2899  1542  2069 ...  4362 13466  5454]
 [    3  3459    53 ...     0     0     0]
 ...
 [   71    14    19 ...   976   107  9625]
 [  845   192    66 ...     0     0     0]
 [13545   605   367 ...     0     0     0]]
[[  328 12069  3133 ...     0     0     0]
 [  250   115    10 ...     3    82  2743]
 [   20   667  4855 ...     0     0     0]
 ...
 [   57   228     2 ...  3584    49   202]
 [    2  4551  4047 ...     0     0     0]
 [   99    33    11 ...     0     0     0]]
